In [2]:
def file_len(fname):
    with open(fname, encoding='utf-8') as f:
        for i, l in enumerate(f):
            pass
    return i + 1
print(file_len('polish_corpora.txt'))

23011601


## Gather words

In [3]:
def gather_words_from_file(file_name, lines_to_read):
    words = {}
    with open(file_name, encoding='utf-8') as f:
        for _ in range(lines_to_read):
            words_in_line = next(f).rstrip('\n').split(' ')
            for word in words_in_line:
                words[word.lower()] = True
    return words

In [4]:
words = gather_words_from_file(file_name='polish_corpora.txt', lines_to_read=500000)

## Exercise 1

### a)

In [5]:
def get_score(tokenization_attempt, proper_tokenization):
    score = 0
    for token in tokenization_attempt:
        if token in proper_tokenization:
            score +=1
    return score / len(proper_tokenization)

In [6]:
get_score(['t1', 't2t3', 't3'], ['t1', 't2', 't3', 't3'])

0.5

### b)

In [7]:
def max_match(text):
    tokens = []
    current_token = ''
    for letter in text:
        extended_token = current_token + letter
        if extended_token in words:
            current_token = extended_token
        else:
            tokens.append(current_token)
            current_token = letter
    if current_token != '':
        tokens.append(current_token)
    return tokens

In [8]:
def tokenize_and_compare(line):
    proper_tokenization = line.lower().split(' ')
    processed_line = "".join(line.split()).lower()
    attempted_tokenization = max_match(processed_line)
    score = get_score(attempted_tokenization, proper_tokenization)
    print(f'expected: {proper_tokenization}')
    print(f'got: {attempted_tokenization}')
    print(f'score: {score}')
    print()

In [9]:
with open('polish_corpora.txt', encoding='utf8') as f:
    for _ in range(5):
        line = next(f).rstrip('\n')
        tokenize_and_compare(line)

expected: ['system', 'euroraty', 'chcesz', 'kupować', 'więcej', 'niż', 'gdzie', 'indziej', '?']
got: ['syst', 'eme', 'uro', 'raty', 'ch', 'ces', 'zk', 'upo', 'wać', 'więc', 'ej', 'niż', 'gd', 'zie', 'ind', 'zie', 'j', '?']
score: 0.2222222222222222

expected: ['parlament', 'zdecydował', 'jednak', 'inaczej', 'i', 'przyjął', 'w', 'ustawie', 'z', 'dnia', '28.06.1996', 'r.', 'jednoinstancyjne', 'postępowanie', 'orzeczniczo-lekarskie', '.']
got: ['par', 'lam', 'en', 'tz', 'dec', 'y', 'dowa', 'ł', 'jednak', 'ina', 'czej', 'ip', 'rzy', 'jął', 'wu', 'stawie', 'zd', 'nia', '28.', '06.', '1996r', '.', 'jedno', 'insta', 'nc', 'yj', 'nep', 'ost', 'ę', 'pow', 'anie', 'orz', 'ec', 'zn', 'ic', 'zo', '-', 'lek', 'ars', 'kie', '.']
score: 0.1875

expected: ['po', 'kampanii', 'wrześniowej', '1.dlek', 'raportowała', '77', 'czołgów', 'l.t.m.35', 'utraconych', '(', 'wraz', 'z', 'wozami', 'dowodzenia', ')', 'i', '52', 'uszkodzone', 'lub', 'zepsute', ',', 'lecz', 'ostatecznie', 'po', 'naprawach', 'straty', '

#### The file is kind of scuffed ngl

In [10]:
with open('polish_corpora.txt', encoding='utf-8') as f:
        for _ in range(500000):
            line = next(f).rstrip('\n')
            words_in_line = line.split(' ')
            if 'zk' in words_in_line:
                print(line)

Nie ch ju ż naw et nie będ zie str a żn ic zk ą , a le n ie ch w o góle b ęd zie .
Moze nie trzeba by bylo interweniowac a miejsce w zk przydaloby sie komus innemu .
A to , czy jakis pies kryl warunkowo czy nie , czy zk jest fajny czy be ...


## Exercise 2

In [14]:
from random import choice


class SentenceGenerator:
    def __init__(self, successors):
        self.successors = successors

    def random_word(self):
        return choice(list(self.successors.keys()))

    def random_successor_of(self, word):
        successors = self.successors[word]
        return choice(successors)[1]

    def has_successors(self, word):
        return word in self.successors and len(self.successors[word]) > 0

    def is_end_of_sentence(self, word):
        return word in ['<EOS>', '.', '?', '!', '?!', '!?']

    def generate_sentence(self):
        word = self.random_word()
        while not self.has_successors(word):
            word = self.random_word()

        sentence = [word]
        while self.has_successors(word):
            successor = self.random_successor_of(word)
            word = self.create_next_word(word, successor)
            sentence.append(successor)

        return ' '.join(sentence) + ('.' if not self.is_end_of_sentence(sentence[-1]) else '')

    def create_next_word(self, current_word, successor):
        _, *rest = current_word.split(' ')
        return ' '.join(rest + [successor])

    def generate_text(self, sentences=1):
        return '\n'.join([self.generate_sentence() for _ in range(sentences)])


In [3]:
def add_successor(successors, n, word, successor):
    successor = (n, successor)
    if word in successors:
        successors[word].append(successor)
    else:
        successors[word] = [successor]

### Bigrams

In [4]:
def gather_successors_from_bigrams(file_name, K=1):
    successors = {}
    with open(file_name, encoding='utf-8') as f:
        line = f.readline()
        while line:
            n, word1, word2 = line.rstrip('\n').split(' ')
            if (int(n) >= K):
                add_successor(successors, int(n), word1, word2)
            line = f.readline()
    return successors

In [5]:
successors2 = gather_successors_from_bigrams('poleval_2grams.txt', 10)

In [18]:
sg2 = SentenceGenerator(successors2)
print(sg2.generate_text(sentences=5))

interferencji w kilka stacji nadawczych oraz zapiski historyczne oddział ratunkowy nie znajduję podstaw aby stany , dąbki . irena turska , dojazdowe i gromadzenie się rośliny posiadają m.in. wspomniane wcześniej 6 mówiący chce podać po konkwiście.
kureń strzelców allsvenskan . ) uczestniczyło 30 500 ochotników strażaków ( ahl w porównianiu z jakiś film posiada ktoś pisze : wschodnia . 49 kobiet zamieszkujących głównie polskiej dopuszcza sie gdzie pojawiały w warszawie-okęciu . była miss dior . 125 mw ) linię pomiędzy 40. kadencji obecnie wersja wydana pozytywna ? <EOS>
kazachstanem i wyróżnione w benasque.
wilanowskiego . bogusław polch.
upakowane , 343 . ponieważ zamek : linię otworzono tu dodawać . barbara bartuś pytała , stabilizując się wychylił się precyzyjnie ocenić przydatność danego przewoźnika do tajgi , mikroklin , torpedy te zniszczenia go zeznania za niemoralne jest potwierdzana przeprowadzeniem w czytelnej i stolarnia i społecznego wykluczenia wykonawcy posiadający dwie li

### Trigrams

In [19]:
def gather_successors_from_trigrams(file_name, K=1):
    successors = {}
    with open(file_name, encoding='utf-8') as f:
        for line in f:
            line = line.rstrip('\n').split(' ')
            if (len(line) != 4): continue
            
            n, word1, word2, word3 = line
            if (int(n) >= K):
                add_successor(successors, int(n), word1, f'{word2} {word3}')
    return successors

In [38]:
successors3 = gather_successors_from_trigrams('poleval_3grams.txt', 5)

In [49]:
sg3 = SentenceGenerator(successors3)
sentence = sg3.generate_text(sentences=5)
print(sentence)

gigli . <EOS>.
archiwalnym i archiwach.
neuroprzekaźników w mózgu.
iwoniczu zdroju ,.
dofinansowywania poszczególnych miast.


In [73]:
for _ in range(10000):
    sentence = sg3.generate_sentence()
    if len(sentence.split(' ')) > 4:
        print(sentence)
print('done')

done


## Exercise 3

In [55]:
import numpy as np

class NonUniformSentenceGenerator(SentenceGenerator):
    def random_successor_of(self, word):
        successors = self.successors[word]
        total_occurences = sum(list(map(lambda s: s[0], successors)))
        probabilites = list(map(lambda s: s[0] / total_occurences, successors))
        successors = list(map(lambda s: s[1], successors))

        return np.random.choice(successors, 1, probabilites)[0]

In [56]:
nusg2 = NonUniformSentenceGenerator(successors2)
print(nusg2.generate_text(sentences=5))

zgodziłoby się nadawało im czci najświętszego imienia siostry miłosierdzia dla regionu podkarpackiego oddziału przedsiębiorstwa małe koszty remontów głównych eksporterów oraz określi . robert biedroń . bernardynów bydgoskich : kwiatostany ( ograniczanie obciążeń nie wiedzieliby , ryk , kierownictwu . xiv jan bloch , tobiasz , jadącego pociągu pasażerskiego ? <EOS>
dwuznaczna . rafał sikora bogatka , dodatkowy , skórne o uzasadnionych przez kwaterę . prout.
łach i kinkiety , przywiązując do cienkich . juliusza . elżbieta dziębowska , avril , australia zachodnia cześć jej powiększenia . 68 minucie po podbiciu . luke . 64 x więcej złotych każdego zwierzęcia na 35 zmiana subwencji przypadającej w hucknall.
dublowane ? <EOS>
cicholską . nice 1 złotoryja . posiada dla samic z nerwami i darem jest propozycji są zestawiane w getyndze i pociągi eic , różewicz , zarejestruj sie jeden więcej poziom odpowiedzialności ... brrr . 1c , dokumentalne o śp. stanisława . katarzyna maria urodziła im prawe

In [64]:
nusg3 = NonUniformSentenceGenerator(successors3)
print(nusg3.generate_text(sentences=5))

murom . <EOS>.
szczepione , lub.
wachlarza możliwości ..
biochemiczny . <EOS>.
przystąpiło 8 zespołów.


## Exercise 4

In [115]:
from itertools import permutations


def compute_score_of_pair(word1, word2, successors):
    if word1 not in successors:
        return 0
    if len(successors[word1]) == 0:
        return 0

    for score, successor in successors[word1]:
        if word2 == successor:
            return score
    
    return 0


def compute_sentence_score(sentence, successors):
    score = 0
    for i in range(len(sentence) - 1):
        word1, word2 = sentence[i:i+2]
        score += compute_score_of_pair(word1, word2, successors)
    return score


def create_sentences(words, successors, n=-1):
    n = len(successors) if n == -1 else n
    perms = list(permutations(words))
    scored = list(
        map(lambda p: (p, compute_sentence_score(p, successors)), perms))
    return sorted(scored, key=lambda s: -s[1])[:n]

#### helper functions for creating bigrams from custom files

In [100]:
from string import punctuation


def remove_punctuation(text):
    return text.translate(str.maketrans('', '', punctuation))


def compute_successors(file_name):
    successors = {}
    with open(file_name, encoding='utf-8') as f:
        for line in f.readlines():
            words = remove_punctuation(line.rstrip('\n')).split(' ')
            for i in range(len(words) - 1):
                word1, word2 = list(map(lambda w: w.lower(), words[i:i+2]))
                if word1 in successors:
                    if word2 in successors[word1]:
                        successors[word1][word2] += 1
                    else:
                        successors[word1][word2] = 1
                else:
                    successors[word1] = {word2: 1}

    scored_successors = {}
    for word in successors:
        scored_successors[word] = []
        for successor in successors[word]:
            score = successors[word][successor]
            scored_successors[word].append((score, successor))

    return scored_successors

In [101]:
computed_successors = compute_successors('test.txt')

#### default successors from bigram file

In [116]:
words = ['judyta', 'poślubiła', 'ottona', 'syna', 'margrabiego']
for sentence in create_sentences(words, successors, 10):
    print(sentence)

(('judyta', 'poślubiła', 'syna', 'ottona', 'margrabiego'), 21)
(('judyta', 'poślubiła', 'margrabiego', 'syna', 'ottona'), 21)
(('judyta', 'syna', 'ottona', 'poślubiła', 'margrabiego'), 21)
(('judyta', 'syna', 'ottona', 'margrabiego', 'poślubiła'), 21)
(('judyta', 'margrabiego', 'poślubiła', 'syna', 'ottona'), 21)
(('judyta', 'margrabiego', 'syna', 'ottona', 'poślubiła'), 21)
(('poślubiła', 'judyta', 'syna', 'ottona', 'margrabiego'), 21)
(('poślubiła', 'judyta', 'margrabiego', 'syna', 'ottona'), 21)
(('poślubiła', 'syna', 'ottona', 'judyta', 'margrabiego'), 21)
(('poślubiła', 'syna', 'ottona', 'margrabiego', 'judyta'), 21)


In [109]:
words = ['nie', 'jest', 'również', 'znana', 'data', 'śmierci', 'Judyty']
for sentence in create_sentences(words, successors, 10):
    print(sentence)

(('znana', 'data', 'śmierci', 'nie', 'jest', 'również', 'Judyty'), 253408)
(('Judyty', 'znana', 'data', 'śmierci', 'nie', 'jest', 'również'), 253408)
(('data', 'śmierci', 'nie', 'jest', 'również', 'znana', 'Judyty'), 253359)
(('Judyty', 'data', 'śmierci', 'nie', 'jest', 'również', 'znana'), 253359)
(('znana', 'Judyty', 'data', 'śmierci', 'nie', 'jest', 'również'), 253285)
(('data', 'śmierci', 'nie', 'jest', 'również', 'Judyty', 'znana'), 253285)
(('śmierci', 'nie', 'jest', 'również', 'znana', 'data', 'Judyty'), 253252)
(('Judyty', 'śmierci', 'nie', 'jest', 'również', 'znana', 'data'), 253252)
(('znana', 'data', 'Judyty', 'śmierci', 'nie', 'jest', 'również'), 253178)
(('śmierci', 'nie', 'jest', 'również', 'Judyty', 'znana', 'data'), 253178)
